非常に参考になる解説：
https://mosko.tokyo/post/pytorch-rnn/

# RNN


In [5]:
%%bash
wget https://github.com/moskomule/pytorch.learning/blob/master/tutorial/rnn.py

wget: /home/toshinao/anaconda3/envs/base2/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /home/toshinao/anaconda3/envs/base2/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /home/toshinao/anaconda3/envs/base2/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-02-09 19:50:12--  https://github.com/moskomule/pytorch.learning/blob/master/tutorial/rnn.py
Resolving github.com (github.com)... 192.30.255.113, 192.30.255.112
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-02-09 19:50:12 ERROR 404: Not Found.



In [2]:
%%bash
git add theoretical_consideraiton.ipynb

In [4]:
%%bash
git commit -a -m "added theoretical_consideration"

[master bbb2aed] added theoretical_consideration
 1 file changed, 143 insertions(+)
 create mode 100644 theoretical_consideraiton.ipynb


$$
\begin{align}
\text{時点$t$における$l$番目の(hidden) layerにおけるnode value}=h^{(l)}_t=f
\left(
    W^{(l)}   
    \begin{bmatrix}
        h^{(l-1)}_t\\
        h^{(l)}_{t-1}
    \end{bmatrix}
\right)
\end{align}
$$


* RNNはoption on optionのように考えると理解しやすい

対応は以下のような感じか？

* ひとつめのオプションに対してクーポンが入力される
    * クーポンはembedded vectorになっている「ワード」なので300次元とかのイメージ 
    * なのでそれを受ける一つ目のオプションも300次元の入力を食えるようになっている
    * 300次元は株価方向のラティスをイメージ？
* ひとつめのオプションバリュー自体も300次元であれば株価イメージでいけるがかならずしも同じ次元である必要はない（よね？）
    * その場合の対応するイメージは？？例えば200次元であれば、オプションンバリューのレイヤーは株価方向を粗視化しているとみなすのがいいか？
        *そうだとするとそれってオプションプライシングでも応用可能かも？


# gated RNN

書籍などの説明では上記のRNNを簡略化したのちにgateを導入しているように見える

soft callのような選択的な期限前償還によりタイムスリップするイメージ
* ただし、満期依存するクーポンでcall notice periodに入るとクーポンのスケジュールもタイムスリップする
* このメカニズムにより非常に遠い満期直前のクーポンの大小が$t=0$におけるオプションバリューに効いてくる


# GRU

gateをたくさんいれたRNN

(以下、gateの名前が書籍と上記リンクで異なるが書籍のほうを採用する)

* gateによりショートカットしてくるバリュー$n$
    * 普通のRNNやgated RNNとほぼ同じものだが、
    * 一時点前の同レイヤーからのチャンネルがreset gate $r$によって遮断されることがある
* update gate $z$はgated RNNのゲートと同様に機能
* update gate $z$およびreset gate $r$は普通のRNNのhidden node valueと同じ形式の演算で決まる
    * 書籍の(2.61)式もそのことを物語っている

## 以下、古い読解

上記文献の場合、
* candidate node $n$が一番素朴なRNN(活性化関数を$\tanh$としたもの)と自然に対応が付くノード
    * ただし、RNNで素直に一時点前の同じレイヤーの隠れ層からの伝搬があるのに対して、GRUではreset gate $r$がゼロの時は伝搬を遮断する
    * そのreset gateは普通のRNNスタイル（活性化関数はsigmoid）のノード
         * ただし、一時点前のreset gateからの伝搬があるわけではなく、通常のhidden node $h$からの伝搬
* update gate $z$が1になるときは　candidate nodeの値は捨てられて、一時点前の同じレイヤーがそのままスライド採用される
    * これはGated RNNに由来する方法か
    

RNNの設計のうち、オプション的な解釈がむずかしいかもしれないところ
* ひとつめのオプションの現在価値と二つ目のオプションの1ステップ前（すなわち未来）とを混合してから活性化関数にかける
    * ひとつめのオプション価値に依存するクーポンと思えばよいか